# Exercise 2: Temporal Hello World — Solution

Complete implementation of the Temporal Hello World workflow with an activity.

## Architecture Pattern 

```
Workflow Execution Request 
    ↓
Temporal Workflow (orchestration) 
    ↓
Temporal Activity (unit of work) 
    ↓
[Process data, call APIs, etc.]
    ↓
Activity completes [OK]
    ↓
Workflow returns result 
    ↓
Return to caller [OK]
```

**Key Benefits:**
- [OK] Activities automatically retry on failure
- [OK] Workflow state persists across crashes
- [OK] Full execution history in Temporal UI

## Setup

Before running this solution, ensure you have:

1. **Temporal Server Running:**
   - Follow `temporal_installation.ipynb` in the project root


In [ ]:
import asyncio
from datetime import datetime, timedelta

import httpx
import nest_asyncio
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import UnsandboxedWorkflowRunner, Worker

## 1. Define an `activity.py`

In [ ]:
@activity.defn
async def get_weather_for_state(state: str) -> str:
    """Activity that fetches real weather alerts from National Weather Service API."""
    # Log activity start with state parameter
    activity.logger.info(f" Fetching weather alerts for {state}")
    
    try:
        # Call the National Weather Service API for active alerts
        url = f"https://api.weather.gov/alerts/active/area/{state.upper()}"
        # Set User-Agent header as required by NWS API
        headers = {"User-Agent": "Temporal-Workshop (educational)"}
        
        # Make async HTTP request with 10 second timeout
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers, timeout=10.0)
            response.raise_for_status()
            
            # Parse JSON response and extract alert features
            data = response.json()
            features = data.get("features", [])
            
            # If no alerts found, return early
            if not features:
                result = f"No active weather alerts for {state.upper()}."
                activity.logger.info("[OK] No active alerts found")
                return result
            
            # Extract and format alert information (limit to first 3)
            alerts = []
            for feature in features[:3]:
                properties = feature.get("properties", {})
                event = properties.get("event", "Unknown")
                severity = properties.get("severity", "Unknown")
                alerts.append(f"- {event} ({severity})")
            
            # Combine results into formatted string
            result = f"Active weather alerts for {state.upper()}:\n" + "\n".join(alerts)
            activity.logger.info(f"[OK] Found {len(features)} alert(s)")
            return result
    
    except httpx.HTTPError as e:
        # Handle HTTP errors (timeouts, connection errors, etc.)
        error_msg = f"Failed to fetch weather alerts: {str(e)}"
        activity.logger.error(f"[ERROR] {error_msg}")
        return error_msg
    except Exception as e:
        # Handle any other unexpected errors
        error_msg = f"Unexpected error: {str(e)}"
        activity.logger.error(f"[ERROR] {error_msg}")
        return error_msg


## 2. Define a `workflow.py`

In [ ]:
@workflow.defn
class HelloWorkflowTemporal:
    """Workflow that orchestrates the weather API activity call."""

    @workflow.run
    async def run(self, state: str) -> str:
        # Log workflow start with state parameter
        workflow.logger.info(f" Workflow started for state: {state}")
        # Execute activity to get weather data with 30 second timeout
        result = await workflow.execute_activity(
            get_weather_for_state,
            args=[state],
            start_to_close_timeout=timedelta(seconds=30),
        )
        # Log workflow completion
        workflow.logger.info("[OK] Workflow finished")
        # Return formatted result to caller
        return f"Workflow result: {result}"

## Create `worker.py`

In [ ]:
async def run_worker():  # Define async function to start and run the worker
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to local Temporal server
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
    )

    # Create worker that polls the task queue for work
    task_queue = "hello-temporal-task-queue"
    worker = Worker(
        client,  # Use the connected Temporal client
        task_queue=task_queue,  # Which queue to poll for tasks
        workflows=[HelloWorkflowTemporal],  # sets the workflow type name in UI
        activities=[get_weather_for_state],  # List of activities this worker can execute
        workflow_runner=UnsandboxedWorkflowRunner(),
    )

    print(f"[OK] Worker started on task queue: {task_queue}")
    print("   Listening for workflow and activity tasks...")
    # Start polling and executing tasks (blocks until stopped)
    await worker.run()


# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()
worker_task = asyncio.create_task(run_worker())
print(" Worker running in background")


## Create `starter.py`

Use the cell below to run the fully implemented solution script.

In [ ]:
async def run_solution():  # Define async function to execute the workflow
    """Execute Temporal workflow to fetch weather alerts."""

    # Generate workflow ID with EST timestamp for human-readable tracking
    est = pytz.timezone("US/Eastern")  # Create EST timezone object
    now = datetime.now(est)  # Get current time in EST
    # Format timestamp as readable string with day-month-date-time pattern
    workflow_id = f"weather-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"

    # Connect to Temporal server
    client = await Client.connect(
        "localhost:7233"  # Local Temporal server address
    )

    print(f" Starting workflow: {workflow_id}")

    # Start the workflow (non-blocking) and get handle for tracking
    task_queue = "hello-temporal-task-queue"
    handle = await client.start_workflow(
        HelloWorkflowTemporal.run,  # Workflow method to execute
        "CA",  # California state code for weather alerts
        id=workflow_id,  # Unique workflow ID for tracking in Temporal UI
        task_queue=task_queue,  # Queue where worker will pick up this workflow
    )

    print(f"[OK] Workflow started: {handle.id}")
    # Print Temporal UI link for observing workflow execution
    print(
        f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )
    # Wait for workflow to complete and get result (blocking)
    result = await handle.result()
    print(f" Workflow completed with result: {result}")

# Run the solution with Jupyter-specific async handling
try:
    # Try to get existing event loop (Jupyter has one running)
    loop = asyncio.get_running_loop()
    # Execute in existing loop
    await run_solution()
except RuntimeError:
    # If no loop exists, create new one and run
    asyncio.run(run_solution())
